In [150]:
from torchtext.vocab import Vectors
import torchtext
import torch.nn as nn
import torch
import numpy as np
import random
import string

In [117]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

# prepare and preprocess data

In [118]:
TEXT = torchtext.data.Field(sequential=True, tokenize=None, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=50, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)



In [153]:
 train, test = torchtext.data.TabularDataset.splits(
        path='./',
        train='train.csv',
        test='test.csv',
        format='csv',
        fields=[('Text', TEXT), ('sentiment', LABEL)])

In [152]:
import nltk
nltk.download('stopwords')
from string import punctuation
from nltk.corpus import stopwords

def punctuation_stopwords_removal(sms):
    # filters charecter-by-charecter : ['h', 'e', 'e', 'l', 'o', 'o', ' ', 'm', 'y', ' ', 'n', 'a', 'm', 'e', ' ', 'i', 's', ' ', 'p', 'u', 'r', 'v', 'a']
    #remove_punctuation = [ch for ch in sms if ch not in punctuation]
    # convert them back to sentences and split into words
    #remove_punctuation = "".join(remove_punctuation).split()
    filtered_sms = [word.lower() for word in sms if word.lower() not in stopwords.words('english')]
    filtered_sms = [word.lower() for word in filtered_sms if word.lower() not in string.punctuation]
    filtered_sms = [word.lower() for word in filtered_sms if word.lower() not in '…']
    return filtered_sms

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\taimo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [154]:
for example in train:
    example.Text=punctuation_stopwords_removal(example.Text)

In [155]:
for example in train:
    print(example.Text, example.sentiment)

['agree', 'poor', 'india', 'treated', 'badly', 'poors', 'seek', 'living', 'singapore', 'treated', 'like', 'citizens', 'given', 'free', 'medical', 'treatment', 'given', 'food', 'daily', 'sim', 'cards', 'call', 'home', 'tell', 'family', 'fine', 'covid', '19', 'case', 'treated', 'foc', 'hospitals'] 0
['could', 'spent', 'cutie', 'vc', 'sakshi__s', 'n', 'g', 'h', 'coast', 'crossing', 'republik'] 1
['nature', 'conservation', 'remain', 'priority', 'post', 'corona', 'bangladesh', 'daily', 'star'] 1
['coronavirus', 'disappearing', 'italy', 'show', '"intellectuals"', 'say', 'lockdowns', 'work'] 0
['uk', 'records', 'lowest', 'daily', 'virus', 'death', 'toll', 'since', 'start', 'lockdown', 'govt'] 0
['joe', "biden's", 'coronavirus', 'web', 'address', 'lands', 'donation', 'page', 'campaign', 'profiting', 'fear', 'demopos'] 0
['respected', 'sir', 'telangana', 'private', 'teachers', 'lecturers', 'horrible', 'situations', 'due', 'covid', '19', 'mainly', 'private', 'management', '(chaithanya', 'nd', 'n

In [156]:
for example in test:
    example.Text=punctuation_stopwords_removal(example.Text)

In [157]:
for example in test:
        print(example.Text, example.sentiment)

['ok', 'well', 'coronavirus', 'entirely', 'different', 'flu'] 0
['breaking', 'uae', 'research', 'institute', 'developed', 'breakthrough', 'treatment', 'covid', '19', 'could', 'game', 'changer', 'global', 'fight', 'virus', '1/5'] 0
['mark', 'words', 'next', 'coronavirus', 'hit', 'list', 'anthony', 'weiner', 'mark', 'damn', 'words'] 0
['manila', 'eases', "world's", 'longest', 'lockdown'] 1
['cross', '5', 'million', 'cases', '2', 'million', 'recoveries', '3', '33', '996', 'deaths', 'virus', 'continues', 'spread', 'adding', '100', '00', 'new', 'cases', 'everyday', 'death', 'toll', 'lat', 'brazil', 'rise', 'still', 'severe', 'rise', 'mortality', 'rate', 'us', 'uk', 'spain', 'italy'] 0
['failed', 'whole', 'nation', 'yetro', '2', 'mahina', 'lockdown', 'rakheko', 'cha', 'gareeb', 'janta', 'lai', 'sakne', 'le', 'ta', 'jasari', 'bachihalchan', 'janata', 'marna', 'baneko', 'loktantra', 'ho', 'yo', 'think', 'neta', 'give', 'two', 'fucks', 'ko', 'bhokai', 'mariracha', 'happily', 'living', 'bubble']

In [158]:
import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける
test_ds, val_ds = test.split(
    split_ratio=0.5, random_state=random.seed(1234))


In [159]:
print('訓練および検証のデータ数', len(train))
print('1つ目の訓練および検証のデータ', vars(train[0]))
print('訓練および検証のデータ数', len(test_ds))
print('1つ目の訓練および検証のデータ', vars(test_ds[0]))

訓練および検証のデータ数 2473
1つ目の訓練および検証のデータ {'Text': ['agree', 'poor', 'india', 'treated', 'badly', 'poors', 'seek', 'living', 'singapore', 'treated', 'like', 'citizens', 'given', 'free', 'medical', 'treatment', 'given', 'food', 'daily', 'sim', 'cards', 'call', 'home', 'tell', 'family', 'fine', 'covid', '19', 'case', 'treated', 'foc', 'hospitals'], 'sentiment': '0'}
訓練および検証のデータ数 309
1つ目の訓練および検証のデータ {'Text': ['wait', 'loved', 'ones', 'fall', 'ill', 'coronavirus'], 'sentiment': '0'}


In [160]:

text_embedding_vectors=Vectors(name="./wiki-news-300d-1M.vec")
print(text_embedding_vectors.dim)
print(len(text_embedding_vectors.itos))

300
999994


In [161]:
TEXT.build_vocab(train, vectors=text_embedding_vectors, min_freq=5)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

torch.Size([1432, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.1197, -0.1732, -0.0780,  ..., -0.0077, -0.0122,  0.0423],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [162]:
TEXT.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x0000026F258B9048>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'coronavirus': 4,
             'corona': 5,
             'covid': 6,
             'lockdown': 7,
             'people': 8,
             '19': 9,
             'virus': 10,
             'twitter': 11,
             'us': 12,
             'pic': 13,
             'get': 14,
             'due': 15,
             'time': 16,
             'one': 17,
             'like': 18,
             'going': 19,
             'please': 20,
             'world': 21,
             'day': 22,
             'home': 23,
             'india': 24,
             'govt': 25,
             "i'm": 26,
             'trump': 27,
             'need': 28,
             'pandemic': 29,
             'cases': 30,
             'go': 31,
             'stay': 32,
             'would': 33,
             'good': 34,
  

In [183]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train, batch_size=5, train=True)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=16, train=False, sort=False)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=16, train=False, sort=False)

In [184]:
batch = next(iter(val_dl))
print(batch.Text[0])
print(batch.sentiment)


tensor([[   2,    5,  845,  107,   35,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   2,   54,    0,    0,    0,   82,  145,    0, 1330, 1177,    0,  351,
          100,  275,   70,    0,    6,    9,   50,    0,  967,  940,    0,   51,
           85,  237,    0,  739,    0,  299,    0,    3,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1],
        [   2,  995,   19,   69,   40,  117,    0,  785,  717,  253,    0,  117,
          692,    0,  272,  422,    8, 1026,  183,  163,    0,  120,    0,    3,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,   

# Defining and testing models 

In [185]:
class Embedder(nn.Module):
    '''idで示されている単語をベクトルに変換します'''

    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()
        
        
        self.embeddings = nn.Embedding.from_pretrained(
            embeddings=text_embedding_vectors, freeze=True)
        # freeze=Trueによりバックプロパゲーションで更新されず変化しなくなります

    def forward(self, x):
        x_vec = self.embeddings(x)

        return x_vec


In [186]:

net1 = Embedder(TEXT.vocab.vectors)

# 入出力
batch = next(iter(train_dl))
x = batch.Text[0]
x1 = net1(x)  # 単語をベクトルに

print("入力のテンソルサイズ：", x.shape)
print("出力のテンソルサイズ：", x1.shape)

入力のテンソルサイズ： torch.Size([5, 50])
出力のテンソルサイズ： torch.Size([5, 50, 300])


In [187]:
class PositionalEncoder(nn.Module):
    '''入力された単語の位置を示すベクトル情報を付加する'''

    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()

        self.d_model = d_model  # 単語ベクトルの次元数

        # 単語の順番（pos）と埋め込みベクトルの次元の位置（i）によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)

        # GPUが使える場合はGPUへ送る、ここでは省略。実際に学習時には使用する
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * (i + 1))/d_model)))

        # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)

        # 勾配を計算しないようにする
        self.pe.requires_grad = False

    def forward(self, x):

        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret


In [188]:
import torch
import math
# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=50)

# 入出力
x = batch.Text[0]
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)

print("入力のテンソルサイズ：", x1.shape)
print("出力のテンソルサイズ：", x2.shape)



入力のテンソルサイズ： torch.Size([5, 50, 300])
出力のテンソルサイズ： torch.Size([5, 50, 300])


In [189]:
class Attention(nn.Module):
    '''Transformerは本当はマルチヘッドAttentionですが、
    分かりやすさを優先しシングルAttentionで実装します'''

    def __init__(self, d_model=300):
        super().__init__()

        # SAGANでは1dConvを使用したが、今回は全結合層で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)

        # Attentionの大きさ調整の変数
        self.d_k = d_model

    def forward(self, q, k, v, mask):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)

        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)

        # ここでmaskを計算
        mask = mask.unsqueeze(1)
        weights = weights.masked_fill(mask == 0, -1e9)

        # softmaxで規格化をする
        normlized_weights = F.softmax(weights, dim=-1)

        # AttentionをValueとかけ算
        output = torch.matmul(normlized_weights, v)

        # 全結合層で特徴量を変換
        output = self.out(output)

        return output, normlized_weights


class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        '''Attention層から出力を単純に全結合層2つで特徴量を変換するだけのユニットです'''
        super().__init__()

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x


In [190]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F 
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()

        # LayerNormalization層
        # https://pytorch.org/docs/stable/nn.html?highlight=layernorm
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        # Attention層
        self.attn = Attention(d_model)

        # Attentionのあとの全結合層2つ
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        # 正規化とAttention
        x_normlized = self.norm_1(x)
        output, normlized_weights = self.attn(
            x_normlized, x_normlized, x_normlized, mask)
        
        x2 = x + self.dropout_1(output)

        # 正規化と全結合層
        x_normlized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normlized2))

        return output, normlized_weights


In [191]:
# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=50)
net3 = TransformerBlock(d_model=300)

# maskの作成
x = batch.Text[0]
input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
input_mask = (x != input_pad)
print(input_mask[0])

# 入出力
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換

print("入力のテンソルサイズ：", x2.shape)
print("出力のテンソルサイズ：", x3.shape)
print("Attentionのサイズ：", normlized_weights.shape)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False])
入力のテンソルサイズ： torch.Size([5, 50, 300])
出力のテンソルサイズ： torch.Size([5, 50, 300])
Attentionのサイズ： torch.Size([5, 50, 50])


In [192]:
class ClassificationHead(nn.Module):
    '''Transformer_Blockの出力を使用し、最後にクラス分類させる'''

    def __init__(self, d_model=300, output_dim=2):
        super().__init__()

        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)  # output_dimはポジ・ネガの2つ

        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, x):
        x0 = x[:, 0, :]  # 各ミニバッチの各文の先頭の単語の特徴量（300次元）を取り出す
        out = self.linear(x0)

        return out


In [193]:
# 動作確認

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net1 = Embedder(TEXT.vocab.vectors)
net2 = PositionalEncoder(d_model=300, max_seq_len=50)
net3 = TransformerBlock(d_model=300)
net4 = ClassificationHead(output_dim=2, d_model=300)

# 入出力
x = batch.Text[0]
x1 = net1(x)  # 単語をベクトルに
x2 = net2(x1)  # Positon情報を足し算
x3, normlized_weights = net3(x2, input_mask)  # Self-Attentionで特徴量を変換
x4 = net4(x3)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力

print("入力のテンソルサイズ：", x3.shape)
print("出力のテンソルサイズ：", x4.shape)


入力のテンソルサイズ： torch.Size([5, 50, 300])
出力のテンソルサイズ： torch.Size([5, 2])


In [194]:

class TransformerClassification(nn.Module):
    '''Transformerでクラス分類させる'''

    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()

        # モデル構築
        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)

    def forward(self, x, mask):
        x1 = self.net1(x)  # 単語をベクトルに
        x2 = self.net2(x1)  # Positon情報を足し算
        x3_1, normlized_weights_1 = self.net3_1(
            x2, mask)  # Self-Attentionで特徴量を変換
        x3_2, normlized_weights_2 = self.net3_2(
            x3_1, mask)  # Self-Attentionで特徴量を変換
        x4 = self.net4(x3_2)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力
        return x4, normlized_weights_1, normlized_weights_2


In [195]:
# 動作確認

# ミニバッチの用意
batch = next(iter(train_dl))

# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=50, output_dim=2)

# 入出力
x = batch.Text[0]
input_mask = (x != input_pad)
out, normlized_weights_1, normlized_weights_2 = net(x, input_mask)

print("出力のテンソルサイズ：", out.shape)
print("出力テンソルのsigmoid：", F.softmax(out, dim=1))


出力のテンソルサイズ： torch.Size([5, 2])
出力テンソルのsigmoid： tensor([[0.4124, 0.5876],
        [0.4167, 0.5833],
        [0.4180, 0.5820],
        [0.3969, 0.6031],
        [0.4389, 0.5611]], grad_fn=<SoftmaxBackward>)


# train and inference

In [196]:
import torch.optim as optim

In [197]:
# モデル構築
net = TransformerClassification(
    text_embedding_vectors=TEXT.vocab.vectors, d_model=300, max_seq_len=50, output_dim=2).to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

# ネットワークの初期化を定義

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        # Liner層の初期化
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
# 訓練モードに設定
net.train()

# TransformerBlockモジュールを初期化実行
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)


print('ネットワーク設定完了')

ネットワーク設定完了


In [198]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

# 最適化手法の設定
learning_rate = 2e-5
optimizer = optim.Adam(net.parameters(), lr=learning_rate)


In [199]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書オブジェクト

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.sentiment.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
                    input_mask = (inputs != input_pad).to(device)

                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net

In [202]:
# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dl, "val": val_dl}
num_epochs = 20
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)



使用デバイス： cuda:0
-----start-------
Epoch 1/20 | train |  Loss: 0.1373 Acc: 0.9462
Epoch 1/20 |  val  |  Loss: 0.3596 Acc: 0.8738
Epoch 2/20 | train |  Loss: 0.1100 Acc: 0.9559
Epoch 2/20 |  val  |  Loss: 0.4016 Acc: 0.8770
Epoch 3/20 | train |  Loss: 0.0951 Acc: 0.9624
Epoch 3/20 |  val  |  Loss: 0.3837 Acc: 0.8835
Epoch 4/20 | train |  Loss: 0.0908 Acc: 0.9624
Epoch 4/20 |  val  |  Loss: 0.4043 Acc: 0.8964
Epoch 5/20 | train |  Loss: 0.0785 Acc: 0.9713
Epoch 5/20 |  val  |  Loss: 0.4543 Acc: 0.8803
Epoch 6/20 | train |  Loss: 0.0617 Acc: 0.9741
Epoch 6/20 |  val  |  Loss: 0.4966 Acc: 0.8770
Epoch 7/20 | train |  Loss: 0.0514 Acc: 0.9822
Epoch 7/20 |  val  |  Loss: 0.4951 Acc: 0.8673
Epoch 8/20 | train |  Loss: 0.0514 Acc: 0.9814
Epoch 8/20 |  val  |  Loss: 0.5270 Acc: 0.8964
Epoch 9/20 | train |  Loss: 0.0422 Acc: 0.9850
Epoch 9/20 |  val  |  Loss: 0.5884 Acc: 0.8803
Epoch 10/20 | train |  Loss: 0.0466 Acc: 0.9838
Epoch 10/20 |  val  |  Loss: 0.5464 Acc: 0.8803
Epoch 11/20 | train |  Lo

In [203]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)

epoch_corrects = 0  # epochの正解数

for batch in (test_dl):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    
    # GPUが使えるならGPUにデータを送る
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.sentiment.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # mask作成
        input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
        input_mask = (inputs != input_pad)

        # Transformerに入力
        outputs, _, _ = net_trained(inputs, input_mask)
        _, preds = torch.max(outputs, 1)  # ラベルを予測

        # 結果の計算
        # 正解数の合計を更新
        epoch_corrects += torch.sum(preds == labels.data)

# 正解率
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(test_dl.dataset),epoch_acc))

テストデータ309個での正解率：0.8997
